# Import libraries

In [1]:
import pandas as pd

# Read datasets

In [2]:
i_data = pd.read_excel('dataset/i_data.xlsx')
ui_data = pd.read_excel('dataset/ui_data.xlsx')

# Evaluation

In [3]:
from src.isne_recommendation import TfidfLinearKernel

In [4]:
model = TfidfLinearKernel.fit(i_data)
prediction = TfidfLinearKernel.predict('Martha Long', i_data, ui_data, model, 10)

In [5]:
train, test = TfidfLinearKernel.train_test_split(ui_data)
evaluation = TfidfLinearKernel.evaluate_model(train, test, i_data, model)
evaluation

0.136986301369863

In [6]:
from src.isne_recommendation import FeatureRatingsKNN

In [7]:
model = FeatureRatingsKNN.fit(ui_data)
prediction = FeatureRatingsKNN.predict('Martha Long', ui_data, model, 10)

In [8]:
train, test = FeatureRatingsKNN.train_test_split(ui_data)
train = train.set_axis(range(len(train)))

In [9]:
train, test = FeatureRatingsKNN.train_test_split(ui_data)
model = FeatureRatingsKNN.fit(train)
evaluation = FeatureRatingsKNN.evaluate_model(train, test, model, 10)
evaluation

0.4349315068493151

In [10]:
from src.isne_recommendation import Hybrid

In [11]:
model = Hybrid.fit(i_data, ui_data)
prediction = Hybrid.predict('Martha Long', i_data, ui_data, model, 10)

In [12]:
train, test = Hybrid.train_test_split(ui_data)
model = Hybrid.fit(i_data, train)
evaluation = Hybrid.evaluate_model(train, test, i_data, model)
evaluation

0.7568493150684932

### Results 

- **The first approach**: _TF-IDF_ and _Linear Kernel_ Performance has an accuracy score of 13.70 percentage
- **The second approach**: _Feature Ratings_ and _KNN Performance_ has an accuracy score of 43.50 percentage
- **The third approach**: _Hybrid_ has an accuracy score of 75.68 percentage